In [42]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os

In [43]:
PATH = 'F:/downlolad2/11_model.pth'  # CHANGE THIS TO YOUR .pth file directory
model = torch.load(PATH, map_location=torch.device('cpu'))

In [44]:
class SymmetricConvAutoencoder(nn.Module):
  def __init__(self, act_fn):
        super(SymmetricConvAutoencoder, self).__init__()

        self.act_fn = act_fn

        # Encoding
        self.enc1 = nn.Conv2d(1, 8, kernel_size=3, stride=2, padding=1)    # Output: 8x256x256
        self.bn1 = nn.BatchNorm2d(8)

        self.enc2 = nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1)   # Output: 16x128x128
        self.bn2 = nn.BatchNorm2d(16)

        self.enc3 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)  # Output: 32x64x64
        self.bn3 = nn.BatchNorm2d(32)

        self.enc4 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)  # Output: 64x32x32
        self.bn4 = nn.BatchNorm2d(64)

        self.enc5 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1) # Output: 128x16x16
        self.bn5 = nn.BatchNorm2d(128)

        self.enc6 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1) # Output: 256x8x8
        self.bn6 = nn.BatchNorm2d(256)

        self.enc7 = nn.Linear(256 * 8 * 8, 2048)                            # Output: 2048
        self.bn7 = nn.BatchNorm1d(2048)

        # Decoding
        self.dec1 = nn.Linear(2048, 256 * 8 * 8)                            # Output: 256x8x8 when reshaped
        self.bn8 = nn.BatchNorm1d(256 * 8 * 8)

        self.dec2 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1) # Output: 128x16x16
        self.bn9 = nn.BatchNorm2d(128)

        self.dec3 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)  # Output: 64x32x32
        self.bn10 = nn.BatchNorm2d(64)

        self.dec4 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)   # Output: 32x64x64
        self.bn11 = nn.BatchNorm2d(32)

        self.dec5 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1)   # Output: 16x128x128
        self.bn12 = nn.BatchNorm2d(16)

        self.dec6 = nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2, padding=1, output_padding=1)    # Output: 8x256x256
        self.bn13 = nn.BatchNorm2d(8)

        self.dec7 = nn.ConvTranspose2d(8, 1, kernel_size=3, stride=2, padding=1, output_padding=1)     # Output: 1x512x512
  def forward(self, x):
    # Encoding
        x1 = self.act_fn(self.bn1(self.enc1(x)))
        x2 = self.act_fn(self.bn2(self.enc2(x1)))
        x3 = self.act_fn(self.bn3(self.enc3(x2)))
        x4 = self.act_fn(self.bn4(self.enc4(x3)))
        x5 = self.act_fn(self.bn5(self.enc5(x4)))
        x6 = self.act_fn(self.bn6(self.enc6(x5)))
        x6_flat = x6.view(-1, 256 * 8 * 8)
        x7 = self.act_fn(self.bn7(self.enc7(x6_flat)))

        # Decoding
        x6_d = self.act_fn(self.bn8(self.dec1(x7)))
        x6_d = x6_d.view(-1, 256, 8, 8)
        x5_d = self.act_fn(self.bn9(self.dec2(x6_d)))   # Skip connection from x6
        #x5_d = x5_d + x5
        x4_d = self.act_fn(self.bn10(self.dec3(x5_d)))  # Skip connection from x5
        #x4_d = x4_d + x4
        x3_d = self.act_fn(self.bn11(self.dec4(x4_d)))  # Skip connection from x4
        #x3_d = x3_d + x3
        x2_d = self.act_fn(self.bn12(self.dec5(x3_d)))  # Skip connection from x3
        #x2_d = x2_d + x2
        x1_d = self.act_fn(self.bn13(self.dec6(x2_d)))  # Skip connection from x2
        x1_d = x1_d + x1
        x_out = self.dec7(x1_d)


        return x_out, x7   # This returns the final stuff and the bottleneck layer

In [45]:
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
from skimage.feature import match_template

act_fn = F.relu
big_model = SymmetricConvAutoencoder(act_fn)
big_model.load_state_dict(model)
symConvAuto = big_model.to(torch.device('cpu'))

# transformation pipeline
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

input_directory = 'F:/Uni Y3/tesi/Giesma'

results_df = pd.DataFrame(columns=['Filename', 'MSE', 'SSIM', 'PSNR', 'MAPE', 'FSIM'])

def calculate_mse(original, reconstructed):
    mse = np.mean((original - reconstructed) ** 2)
    return mse

def calculate_mape(original, reconstructed, epsilon=1e-10):
    abs_diff = np.abs(original - reconstructed)
    mape = np.mean(abs_diff / (original + epsilon)) * 100
    return mape

def calculate_fsim(original, reconstructed):
    result = match_template(original, reconstructed)
    fsim = np.max(result)
    return fsim

def calculate_ssim(original, reconstructed):
    ssim_value = structural_similarity(original, reconstructed, data_range=reconstructed.max() - reconstructed.min())
    return ssim_value

for filename in os.listdir(input_directory):
    if filename.endswith('.png'):
        image_path = os.path.join(input_directory, filename)
        
        try:
            image = Image.open(image_path)
            image = transform(image)
            
            big_model.eval()
            
            with torch.no_grad():
                reconstructed_pic, _ = symConvAuto(image.unsqueeze(0))
           
            reconstructed_np = reconstructed_pic.squeeze().detach().numpy()
            initial_np = image.squeeze().cpu().numpy()
            
            #calc metrics
            mse = calculate_mse(initial_np, reconstructed_np)
            ssim_value = calculate_ssim(initial_np, reconstructed_np)
            psnr_value = peak_signal_noise_ratio(initial_np, reconstructed_np, data_range=reconstructed_np.max() - reconstructed_np.min())
            mape = calculate_mape(initial_np, reconstructed_np)
            fsim_value = calculate_fsim(initial_np, reconstructed_np)
            
            # append res
            results_df = pd.concat([results_df, pd.DataFrame({
                'Filename': [filename],
                'MSE': [mse],
                'SSIM': [ssim_value],
                'PSNR': [psnr_value],
            })], ignore_index=True)
        
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# calculate averages and append
average_metrics = results_df.drop(columns=['Filename']).mean(axis=0)
average_metrics['Filename'] = 'Average'
results_df = pd.concat([results_df, pd.DataFrame(average_metrics).T], ignore_index=True)

output_excel_path = 'F:/Uni Y3/tesi/Giesma/giesma.xlsx'
results_df.to_excel(output_excel_path, index=False)

print(f"Results saved to {output_excel_path}")

C:\Users\marco\AppData\Local\Temp\ipykernel_22416\4256437668.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({


Results saved to F:/Uni Y3/tesi/Giesma/giesma.xlsx
